# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import pickle
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
import re
import numpy as np

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse_new.db')
df = pd.read_sql_table('df',engine)
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [3]:
# load feature set (X) and label set (Y)
X = df['message'] 
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# custom transformer
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [6]:
# pipeline that uses feature union
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [7]:
# split X and y into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# model predictions
y_pred = pipeline.predict(X_test)

In [9]:
# calculate accuracy score for predictions
accuracy = (y_pred == y_test).mean()

print("Accuracy:")
print(accuracy)
print("Overall Accuracy:", accuracy.mean())

Accuracy:
related                   0.729853
request                   0.831197
offer                     0.994811
aid_related               0.556929
medical_help              0.916209
medical_products          0.945055
search_and_rescue         0.969017
security                  0.982448
military                  0.966270
child_alone               1.000000
water                     0.928877
food                      0.885226
shelter                   0.907509
clothing                  0.982448
money                     0.976648
missing_people            0.990232
refugees                  0.962912
death                     0.947650
other_aid                 0.859127
infrastructure_related    0.934066
transport                 0.955128
buildings                 0.946276
electricity               0.979090
tools                     0.994353
hospitals                 0.990232
shops                     0.995421
aid_centers               0.988553
other_infrastructure      0.954212
weather_re

In [11]:
# convert numpy array to dataframe
y_pred = pd.DataFrame(y_pred, columns=y_test.columns)

# scores report
for col in y_test.columns:
    print('Attribute: {}\n'.format(col))
    print(classification_report(y_test[col], y_pred[col]))

Attribute: related

             precision    recall  f1-score   support

        0.0       0.30      0.11      0.17      1475
        1.0       0.77      0.92      0.84      5023
        2.0       0.00      0.00      0.00        54

avg / total       0.66      0.73      0.68      6552

Attribute: request

             precision    recall  f1-score   support

        0.0       0.84      0.98      0.90      5438
        1.0       0.43      0.07      0.12      1114

avg / total       0.77      0.83      0.77      6552

Attribute: offer

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6521
        1.0       0.00      0.00      0.00        31

avg / total       0.99      1.00      0.99      6552

Attribute: aid_related

             precision    recall  f1-score   support

        0.0       0.58      0.83      0.69      3800
        1.0       0.44      0.19      0.26      2752

avg / total       0.53      0.56      0.51      6552

Attri

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
# old parameters for grid search 
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000), 
        'features__text_pipeline__tfidf__use_idf': (True, False)
    }

# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters)

In [12]:
# fit dataset to grid search
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)), 'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0), 'features__text_pipeline__vect__max_features': (None, 5000, 10000), 'features__text_pipeline__tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
# model predictions
y_pred_new = cv.predict(X_test)

In [21]:
# Overall accuracy of improved model
accuracy = (y_pred_new == y_test).mean()
print("Overall Accuracy:", accuracy.mean())

Overall Accuracy: 0.9251500814


In [14]:
# convert numpy array to dataframe
y_pred_new = pd.DataFrame(y_pred_new, columns=y_test.columns)

# scores report
for col in y_test.columns:
    print('Attribute: {}\n'.format(col))
    print(classification_report(y_test[col], y_pred_new[col]))

Attribute: related

             precision    recall  f1-score   support

        0.0       0.37      0.09      0.14      1499
        1.0       0.77      0.96      0.85      5005
        2.0       0.33      0.02      0.04        48

avg / total       0.68      0.75      0.69      6552

Attribute: request

             precision    recall  f1-score   support

        0.0       0.83      0.99      0.90      5428
        1.0       0.39      0.04      0.07      1124

avg / total       0.76      0.82      0.76      6552

Attribute: offer

             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      6518
        1.0       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6552

Attribute: aid_related

             precision    recall  f1-score   support

        0.0       0.59      0.87      0.71      3824
        1.0       0.47      0.16      0.24      2728

avg / total       0.54      0.58      0.51      6552

Attri

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# Use AdaBoost instead of RandomForest

In [8]:
# new pipeline that uses feature union and AdaBoost as classifier
pipeline_new = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [9]:
# modified parameters for grid search to expedite training process
parameters_new = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75),
        'features__text_pipeline__vect__max_features': (None, 5000), 
        'features__text_pipeline__tfidf__use_idf': (True, False)
    }

# create grid search object
cv_ada = GridSearchCV(pipeline_new, param_grid=parameters_new)

In [10]:
# fit dataset to grid search
cv_ada.fit(X_train,   y_train) 

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)), 'features__text_pipeline__vect__max_df': (0.5, 0.75), 'features__text_pipeline__vect__max_features': (None, 5000), 'features__text_pipeline__tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [11]:
# model predictions
y_pred_ada = cv_ada.predict(X_test)

# Overall accuracy of improved model
accuracy = (y_pred_ada == y_test).mean()
print("Overall Accuracy:", accuracy.mean())

Overall Accuracy: 0.926188780355


In [12]:
# convert numpy array to dataframe
y_pred_ada = pd.DataFrame(y_pred_ada, columns=y_test.columns)

# scores report
for col in y_test.columns:
    print('Attribute: {}\n'.format(col))
    print(classification_report(y_test[col], y_pred_ada[col]))

Attribute: related

             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00      1602
        1.0       0.75      1.00      0.86      4909
        2.0       0.43      0.07      0.12        41

avg / total       0.56      0.75      0.64      6552

Attribute: request

             precision    recall  f1-score   support

        0.0       0.85      0.97      0.90      5463
        1.0       0.45      0.14      0.21      1089

avg / total       0.78      0.83      0.79      6552

Attribute: offer

             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      6517
        1.0       0.00      0.00      0.00        35

avg / total       0.99      0.99      0.99      6552

Attribute: aid_related

             precision    recall  f1-score   support

        0.0       0.59      0.92      0.72      3838
        1.0       0.48      0.11      0.18      2714

avg / total       0.55      0.58      0.50      6552

Attri

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [14]:
# save the model as a pickle file
filename = 'classifier.sav'
pickle.dump(cv_ada, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.